In [1]:
from pathlib import Path


from matplotlib import rcParams
from matplotlib.cm import rainbow
import math
import numpy as np
import pandas as pd


from math import sin, cos, sqrt, atan2, radians


from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn import  linear_model

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
# from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)


pd.set_option("display.max_columns", 100)




/kaggle/input/ml-dataset/ML data/train.csv
/kaggle/input/ml-dataset/ML data/test.csv
/kaggle/input/ml-dataset/ML data/sample_submission.csv
/kaggle/input/fare-classification/meta_data.csv


Definition

In [2]:
R = 6373.0

#lable encording
def encoding_label(label):
  if(label=='correct'):
    return 1
  else:
    return 0
#calculate hours
def calculate_hours(time):
    hours=time.total_seconds()
    return hours    
#calculate distance
def calculate_distance(coordinates):

#     print(coordinates['pick_lat'])
#     print(type(coordinates['pick_lat']))
    lat1 = math.radians(coordinates['pick_lat'])
    lon1 = math.radians(coordinates['pick_lon'])
    lat2 = math.radians(coordinates['drop_lat'])
    lon2 = math.radians(coordinates['drop_lon'])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance
        




In [3]:

train_data = pd.read_csv("/kaggle/input/ml-dataset/ML data/train.csv", index_col="tripid")
train_data["label"]=train_data["label"].apply(encoding_label).values
test_data = pd.read_csv("/kaggle/input/ml-dataset/ML data/test.csv", index_col="tripid")
submissionFormatData = pd.read_csv("/kaggle/input/ml-dataset/ML data/sample_submission.csv", index_col="tripid")
#training_data=train_data["label"]
# index_data=test_data["tripid"]
# train_data.head()
# Trainng_feature=train_data[]
# trainng_feature= train_data.loc[:,[0:12]]


# trainng_label.head()
# test_data.head()
# train_data.columns.values
# train_data.dropna()
# train_data.dropna()

Data Preprocessing 

In [4]:
#feature engineering

#Reemove NAN tuples
train_data.dropna(inplace=True)
#dropNAN columns from test list
NANcolumns=test_data.columns[test_data.isna().any()].tolist()
#remove tuples from submissionData
submissionFormatData.drop(NANcolumns)
test_data.dropna(inplace=True)

trainng_label= train_data[["label"]]
trainng_feature= train_data.loc[:,"additional_fare":"fare"]


trainng_feature['pickup_time'] = pd.to_datetime(trainng_feature['pickup_time'],format='%m/%d/%Y %H:%M')
trainng_feature['drop_time'] = pd.to_datetime(trainng_feature['drop_time'],format='%m/%d/%Y %H:%M')
test_data['pickup_time'] = pd.to_datetime(test_data['pickup_time'],format='%m/%d/%Y %H:%M')
test_data['drop_time'] = pd.to_datetime(test_data['drop_time'],format='%m/%d/%Y %H:%M')


trainng_feature['time_difference']=trainng_feature['drop_time']-trainng_feature['pickup_time']
trainng_feature['time_difference']=trainng_feature['time_difference'].apply(calculate_hours).values
test_data['time_difference']=test_data['drop_time']-test_data['pickup_time']
test_data['time_difference']=test_data['time_difference'].apply(calculate_hours).values

#calculate distance

# trainng_feature['time_difference']=trainng_feature['time_difference'].days
# trainng_feature['pyear']=trainng_feature['pickup_time'].dt.year 
# trainng_feature['pmonth']=trainng_feature['pickup_time'].dt.month 
# trainng_feature['pday']=trainng_feature['pickup_time'].dt.day
# trainng_feature['pdayofweek_num']=trainng_feature['pickup_time'].dt.dayofweek 
# trainng_feature['pHour'] = trainng_feature['pickup_time'].dt.hour 
# trainng_feature['pminute'] = trainng_feature['pickup_time'].dt.minute 




trainng_feature=trainng_feature.drop(['pickup_time','drop_time'],axis=1)
test_data=test_data.drop(['pickup_time','drop_time'],axis=1)
# trainng_feature['dyear']=trainng_feature['drop_time'].dt.year 
# trainng_feature['dmonth']=trainng_feature['drop_time'].dt.month 
# trainng_feature['dday']=trainng_feature['drop_time'].dt.day
# trainng_feature['ddayofweek_num']=trainng_feature['drop_time'].dt.dayofweek 
# trainng_feature['dHour'] = trainng_feature['drop_time'].dt.hour 
# trainng_feature['dminute'] = trainng_feature['drop_time'].dt.minute 
# test_data.isnull().sum()
#merginh two dataframes
trainng_feature["distance"]=trainng_feature[['pick_lat','pick_lon','drop_lat','drop_lon']].apply(calculate_distance,axis = 1)
test_data["distance"]=test_data[['pick_lat','pick_lon','drop_lat','drop_lon']].apply(calculate_distance,axis = 1)
# drop columns
trainng_feature=trainng_feature.drop(['pick_lat','pick_lon','drop_lat','drop_lon'],axis=1)
test_data=test_data.drop(['pick_lat','pick_lon','drop_lat','drop_lon'],axis=1)
trainng_feature.head()
# trainng_feature.isnull().sum()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,time_difference,distance
tripid,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,270.32,840.0,5.094369
189125358,10.5,791.0,47.0,0.0000,134.0,197.85,780.0,3.169052
189125719,10.5,1087.0,80.0,0.0000,61.0,301.64,1080.0,6.307375
189127273,10.5,598.0,271.0,15.6638,68.0,82.30,600.0,0.862217
189129552,10.5,3407.0,182.0,0.0000,112.0,1065.02,3420.0,24.214638


In [5]:
# numerical_cols = trainng_feature.columns[trainng_feature.dtypes != "object"].values

In [6]:
# numerical_cols

In [7]:
# numeric_preprocessing_steps = Pipeline([
#     ('standard_scaler', StandardScaler()),
#     ('simple_imputer', SimpleImputer(strategy='mean'))
# ])

# preprocessor = ColumnTransformer(
#     transformers = [
#         ("numeric", numeric_preprocessing_steps, numerical_cols)
#     ],
#     remainder = "drop"
# )

In [8]:
estimators = MultiOutputClassifier(
    estimator=RandomForestClassifier(max_depth=30,n_estimators=2000,max_features='sqrt',bootstrap=False,min_samples_leaf=1,min_samples_split=10)
)

In [9]:
full_pipeline = Pipeline([
#     ("preprocessor", preprocessor),
    ("estimators", estimators),
])

In [10]:
# full_pipeline

Creating the dataset

In [11]:
X_train, X_eval, y_train, y_eval = train_test_split(
    trainng_feature,
    trainng_label,
    test_size=0.3,
    shuffle=True,
    stratify= trainng_label,
    random_state=6
)
X_train.head()

# y_train.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,time_difference,distance
tripid,,,,,,,,
210059114,10.5,137.0,14.0,0.0,108.0,60.50,120.0,2.703705
210185622,10.5,1003.0,72.0,0.0,53.0,266.07,1020.0,7.312769
211462282,10.5,2140.0,161.0,0.0,42.0,459.45,2100.0,12.058562
197163549,10.5,1013.0,205.0,0.0,24.0,189.74,1020.0,4.841382
201090781,10.5,1213.0,76.0,0.0,103.0,298.92,1260.0,5.423882


Train the model

In [12]:
full_pipeline.fit(X_train, y_train)

# Predict on evaluation set
# This competition wants probabilities, not labels
preds = full_pipeline.predict(X_eval)
preds

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [0]])

In [13]:
print("test_probas[0].shape", preds.shape)

test_probas[0].shape (5091, 1)


In [14]:
y_preds = pd.DataFrame(
    {
        "prediction": preds[:, 0]
    },
    index = y_eval.index
)
print("y_preds.shape:", y_preds.shape)
y_preds.head()

y_preds.shape: (5091, 1)


,prediction
tripid,
194208836,1
212388520,1
204027455,1
196526793,0
195624613,1


Test model

In [15]:
f1_score(y_eval, y_preds, average=None)

array([0.66666667, 0.97349449])

In [16]:
full_pipeline.fit(trainng_feature, trainng_label)

Pipeline(memory=None,
         steps=[('estimators',
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=False,
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                        max_depth=30,
                                                                        max_features='sqrt',
                                                                        max_leaf_nodes=None,
                                                                        max_samples=None,
                                                                        min_impurity_decrease=0.0,
                                                                        min_impurity_split=None,
                                

In [17]:
test_probas = full_pipeline.predict(test_data)
test_probas

array([[1],
       [0],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [18]:
# Make sure we have the rows in the same order
np.testing.assert_array_equal(test_data.index.values, 
                              submissionFormatData.index.values)


submissionFormatData["prediction"]=test_probas[:, 0]
submissionFormatData.head()

submissionFormatData.to_csv('my_submission.csv', index=True)

Show output

In [19]:
!head my_submission.csv

tripid,prediction
213284604,1
213286352,0
213293973,1
213294622,1
213298687,1
213299545,0
213302332,1
213302671,1
213305594,1
